# exercise with pandas

every "..." should be replaced with a line of code

In [1]:
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import os
import io
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as plt
from pandas import Series, DataFrame
import json
from pandas.io.json import json_normalize
import sys
import scipy as sp

%pylab inline
import datetime


puidata= os.getenv("PUIDATA")
if puidata is None:
    os.environ["PUIDATA"] = "%s/PUIdata"%os.getenv("HOME")

try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
    
from sklearn.decomposition import PCA
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sma
import patsy
from statsmodels.graphics.api import abline_plot
import numpy.linalg as linalg
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
sns.set(context='notebook', style='whitegrid', palette='deep', font='sans-serif', font_scale=1, rc=None)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Populating the interactive namespace from numpy and matplotlib


Read what the PLUTO data is [here](https://www.wired.com/2013/08/nyc-pluto-data-map-party/)

Find the PLUTO dataset online, find the cvs file data (not the shape file), it should be about 46M

Download it as a zip file, unpack it in PUIDATAEach borough will be in a different file. The goal is to find land plot categories of which there is a higher number in Manhattan (MN) than in Brooklyn (BK)

Read in csv data as pandas dataframes (with pd.read_csv() )

In a Pandas DataFrame each column is a _series_ (a pandas datatype) to manipulate the data here you will mostly use methods (functions that are defined for a specific datatype and invoked with the _object.method()_ syntax) of the DataFrames (e.g. df.groupby() ) or of the series therein (e.g. series.astype() )

In [2]:
def getPLUTO(datestring):
    print ("Downloading")
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "MN_18v1.csv"):
        if os.path.isfile("MN_18v1.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "MN_18v1.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + "nyc_pluto_18v1.zip"):
                if not os.path.isfile("nyc_pluto_18v1.zip"):
                    os.system("curl -O https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_18v1.zip" + "nyc_pluto_18v1.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + "nyc_pluto_18v1.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + "nyc_pluto_18v1.zip")
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" +  "MN_18v1.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

In [3]:
# MN = pd.read_csv(os.getenv("PUIDATA") + "/" + "MN_18v1.csv", sep=',', skiprows=0, header=0, index_col=None, engine='python')
path1 = os.getenv("PUIDATA") + "/" + "MN_18v1.csv"
path2 = os.getenv("PUIDATA") + "/" + "BK_18v1.csv"
MN = pd.read_csv(path1, skiprows=0, delim_whitespace=False, header=0, index_col=None, engine='python')
BK = pd.read_csv(path2, skiprows=0, delim_whitespace=False, header=0, index_col=None, engine='python')

In [4]:
type(MN)

pandas.core.frame.DataFrame

# read in te Manhattan and Brooklyn data in 2 separate dataframes w pandas

## show the first 10 lines of each df

# show manhattan 10 rows


In [5]:
MN.head(10)

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_Flag,PFIRM15_Flag,Version
0,MN,1,10,101,5.0,1011.0,2.0,1.0,10004.0,E007,...,Y,199 999,10101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1
1,MN,1,101,101,1.0,1001.0,2.0,1.0,10004.0,E007,...,Y,199 999,10101.0,NaN,0.000000e+00,NaN,1,NaN,1.0,18V1
2,MN,1,201,101,1.0,1000.0,2.0,1.0,10004.0,E007,...,NaN,199 999,10101.0,NaN,0.000000e+00,NaN,1,NaN,1.0,18V1
3,MN,1,301,101,NaN,NaN,2.0,1.0,10004.0,E007,...,NaN,199 999,10101.0,NaN,0.000000e+00,NaN,4,1.0,1.0,18V1
4,MN,1,401,101,NaN,NaN,2.0,1.0,10004.0,E007,...,NaN,1 99 999,10101.0,NaN,0.000000e+00,NaN,4,1.0,1.0,18V1
5,MN,2,1,101,9.0,1025.0,2.0,1.0,10004.0,L015,...,NaN,101N100,10101.0,NaN,1.000020e+09,08/07/2012,1,1.0,1.0,18V1
6,MN,2,2,101,9.0,1025.0,2.0,1.0,10004.0,L015,...,NaN,11N 100,10101.0,NaN,1.000020e+09,08/07/2012,1,1.0,1.0,18V1
7,MN,2,3,101,9.0,NaN,2.0,1.0,10004.0,L015,...,NaN,101N100,10101.0,NaN,1.000020e+09,08/07/2012,1,1.0,1.0,18V1
8,MN,2,23,101,9.0,1025.0,2.0,1.0,10004.0,L015,...,NaN,101N100,10101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1
9,MN,3,1,101,319.0,1000.0,2.0,1.0,10004.0,L015,...,NaN,101S087,10101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1


In [6]:
# show brooklyn 10 rows
BK.head(10)

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version
0,BK,1,1,302,21.0,NaN,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,3.000010e+09,11/26/2013,1,1.0,1.0,18V1
1,BK,1,50,302,21.0,2000.0,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,E-231,0.000000e+00,NaN,1,1.0,1.0,18V1
2,BK,1,7501,302,21.0,2000.0,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,3.000010e+09,03/04/2016,1,1.0,1.0,18V1
3,BK,3,1,302,21.0,3002.0,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1
4,BK,3,5,302,21.0,NaN,13.0,33.0,11201.0,L118,...,NaN,302 007,30101.0,NaN,0.000000e+00,NaN,4,1.0,1.0,18V1
5,BK,3,35,302,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,302 007,3.0,NaN,NaN,NaN,5,NaN,NaN,18V1
6,BK,5,1,302,21.0,3001.0,13.0,33.0,11201.0,L118,...,NaN,302 008,30101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1
7,BK,5,8,302,21.0,NaN,13.0,33.0,11201.0,L118,...,NaN,300 000,30101.0,NaN,0.000000e+00,NaN,4,1.0,1.0,18V1
8,BK,6,1,302,21.0,3000.0,13.0,33.0,11201.0,E207,...,NaN,302 009,30101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1
9,BK,6,10,302,21.0,NaN,13.0,33.0,11201.0,E207,...,NaN,302 009,30101.0,NaN,0.000000e+00,NaN,1,1.0,1.0,18V1


## display all columns in each file

In [7]:
# show bk
# Ref: https://stackoverflow.com/questions/24901766/python-how-to-get-column-names-from-pandas-dataframe-but-only-for-continuous
HeadersBK = BK.dtypes.index
HeadersBK

Index([u'Borough', u'Block', u'Lot', u'CD', u'CT2010', u'CB2010',
       u'SchoolDist', u'Council', u'ZipCode', u'FireComp', u'PolicePrct',
       u'HealthCenterDistrict', u'HealthArea', u'SanitBoro', u'SanitDistrict',
       u'SanitSub', u'Address', u'ZoneDist1', u'ZoneDist2', u'ZoneDist3',
       u'ZoneDist4', u'Overlay1', u'Overlay2', u'SPDist1', u'SPDist2',
       u'SPDist3', u'LtdHeight', u'SplitZone', u'BldgClass', u'LandUse',
       u'Easements', u'OwnerType', u'OwnerName', u'LotArea', u'BldgArea',
       u'ComArea', u'ResArea', u'OfficeArea', u'RetailArea', u'GarageArea',
       u'StrgeArea', u'FactryArea', u'OtherArea', u'AreaSource', u'NumBldgs',
       u'NumFloors', u'UnitsRes', u'UnitsTotal', u'LotFront', u'LotDepth',
       u'BldgFront', u'BldgDepth', u'Ext', u'ProxCode', u'IrrLotCode',
       u'LotType', u'BsmtCode', u'AssessLand', u'AssessTot', u'ExemptLand',
       u'ExemptTot', u'YearBuilt', u'YearAlter1', u'YearAlter2', u'HistDist',
       u'Landmark', u'BuiltFAR', u'

In [8]:
# show mn
HeadersMN = MN.dtypes.index
HeadersMN

Index([u'Borough', u'Block', u'Lot', u'CD', u'CT2010', u'CB2010',
       u'SchoolDist', u'Council', u'ZipCode', u'FireComp', u'PolicePrct',
       u'HealthCenterDistrict', u'HealthArea', u'SanitBoro', u'SanitDistrict',
       u'SanitSub', u'Address', u'ZoneDist1', u'ZoneDist2', u'ZoneDist3',
       u'ZoneDist4', u'Overlay1', u'Overlay2', u'SPDist1', u'SPDist2',
       u'SPDist3', u'LtdHeight', u'SplitZone', u'BldgClass', u'LandUse',
       u'Easements', u'OwnerType', u'OwnerName', u'LotArea', u'BldgArea',
       u'ComArea', u'ResArea', u'OfficeArea', u'RetailArea', u'GarageArea',
       u'StrgeArea', u'FactryArea', u'OtherArea', u'AreaSource', u'NumBldgs',
       u'NumFloors', u'UnitsRes', u'UnitsTotal', u'LotFront', u'LotDepth',
       u'BldgFront', u'BldgDepth', u'Ext', u'ProxCode', u'IrrLotCode',
       u'LotType', u'BsmtCode', u'AssessLand', u'AssessTot', u'ExemptLand',
       u'ExemptTot', u'YearBuilt', u'YearAlter1', u'YearAlter2', u'HistDist',
       u'Landmark', u'BuiltFAR', u'

# drop all columns but ZipCode and LandUse

In [9]:
# drop columns here. Use the .drop method of pandas dataframes, i.e. something like bk.drop()... 
# make sure you drop "in place" and that you drop along the right axis - i.e. the columns. 
# you can use bk.drop? to the help with this function 
#Ref: https://stackoverflow.com/questions/16616141/deleting-all-columns-except-a-few-python-pandas
MN_ZCLU = ['ZipCode', 'LandUse']
MN1 = MN[MN_ZCLU]
MN1.head()

,ZipCode,LandUse
0,10004.0,8.0
1,10004.0,8.0
2,10004.0,NaN
3,10004.0,7.0
4,10004.0,7.0


In [10]:
# drop columns here...
BK_ZCLU = ['ZipCode', 'LandUse']
BK1 = BK[BK_ZCLU]
BK1.head()
BK1.head()

,ZipCode,LandUse
0,11201.0,11.0
1,11201.0,5.0
2,11201.0,4.0
3,11201.0,7.0
4,11201.0,7.0


# use groupby to count the building per LandUse code 

In [11]:
# group Manhattan by land use. Make sure you understand how to use the groupby() method:
# this method returns a group object. those objects have methods and some of these methods return dataframes. 
# What do you want to do? you want to count all objects per each land use type. 
# So you want to use .groupby() to group by LandUse and then .count() to get the number of element per LandUse
# Other methods include .mean() to get the mean of a group, .std(), 
# and a generic method .apply() to which you can pass any function that operates on a column.
# All these methods return dataframes 
# (hint: use also reset_index) for each df

# Ref: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html

MNGrouped = MN1.groupby(['LandUse'], axis=0, level=None, as_index=False, sort=True, group_keys=True, squeeze=False).count()
MNGrouped

,LandUse,ZipCode
0,1.0,4019
1,2.0,16246
2,3.0,4881
3,4.0,7238
4,5.0,4465
5,6.0,726
6,7.0,386
7,8.0,2456
8,9.0,434
9,10.0,679


In [12]:
# group BK by land use
BKGrouped = BK1.groupby(['LandUse'], axis=0, level=None, as_index=False, sort=True, group_keys=True, squeeze=False).count()
BKGrouped

,LandUse,ZipCode
0,1.0,156271
1,2.0,66358
2,3.0,3437
3,4.0,22515
4,5.0,5743
5,6.0,4434
6,7.0,1040
7,8.0,4330
8,9.0,872
9,10.0,4229


# convert the landuse to integer 

In [13]:
# convert LandUse to int using the .astype() method of the series
# Ref: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html
# Ref to second example
# BK_Convert1 = BKGrouped.Series(['LandUse'])
BK_Int_Converted = BKGrouped.astype('int', copy=False)
BK_Int_Converted.head(5)

,LandUse,ZipCode
0,1,156271
1,2,66358
2,3,3437
3,4,22515
4,5,5743


In [14]:
MN_Int_Converted = MNGrouped.astype('int', copy=False)
MN_Int_Converted.head(5)

,LandUse,ZipCode
0,1,4019
1,2,16246
2,3,4881
3,4,7238
4,5,4465


# merge the dataframes on land use 

In [15]:
# merge the dataframes on LandUse. You can use the pandas function merge ( _pd.merge)
# Ref: https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
MN_BK_Merged = BK_Int_Converted.merge(MN_Int_Converted, on = 'LandUse', how='outer')
MN_BK_Merged.columns


Index([u'LandUse', u'ZipCode_x', u'ZipCode_y'], dtype='object')

In [16]:
MN_BK_Merged

,LandUse,ZipCode_x,ZipCode_y
0,1,156271,4019
1,2,66358,16246
2,3,3437,4881
3,4,22515,7238
4,5,5743,4465
5,6,4434,726
6,7,1040,386
7,8,4330,2456
8,9,872,434
9,10,4229,679


# rename things so that you have descriptive names

## rename the columns with descriptive names. 


In [17]:
# Use the df.rename() method (hint: it takes a dictionary)
# now you can just call "both" instead of "both.head()" since the file should only have 11 columns
# index=str, 
MN_BK_Merged_ColumnsRenamed = MN_BK_Merged.rename(columns={"ZipCode_x": "bk", "ZipCode_y": "mn"})
MN_BK_Merged_ColumnsRenamed


,LandUse,bk,mn
0,1,156271,4019
1,2,66358,16246
2,3,3437,4881
3,4,22515,7238
4,5,5743,4465
5,6,4434,726
6,7,1040,386
7,8,4330,2456
8,9,872,434
9,10,4229,679


## find online the LandUse data dictionary and replace code w the verbose description of it


In [18]:
# set up Land Use values to descriptive tag
# https://stackoverflow.com/questions/13842088/set-value-for-particular-cell-in-pandas-dataframe-using-index
MN_BK_Merged_ColumnsRenamedDescribed0 = MN_BK_Merged_ColumnsRenamed.set_value(0, ['LandUse'], 'One &Two Family Buildings')
MN_BK_Merged_ColumnsRenamedDescribed1 = MN_BK_Merged_ColumnsRenamed.set_value(1, ['LandUse'], 'Multi-Family Walk-Up Buildings')
MN_BK_Merged_ColumnsRenamedDescribed2 = MN_BK_Merged_ColumnsRenamed.set_value(2, ['LandUse'], 'Multi-Family Elevator Buildings')
MN_BK_Merged_ColumnsRenamedDescribed3 = MN_BK_Merged_ColumnsRenamed.set_value(3, ['LandUse'], 'Mixed Residential & Commercial Buildings')
MN_BK_Merged_ColumnsRenamedDescribed4 = MN_BK_Merged_ColumnsRenamed.set_value(4, ['LandUse'], 'Commercial & Office Buildings')
MN_BK_Merged_ColumnsRenamedDescribed5 = MN_BK_Merged_ColumnsRenamed.set_value(5, ['LandUse'], 'Industrial & Manufacturing')
MN_BK_Merged_ColumnsRenamedDescribed6 = MN_BK_Merged_ColumnsRenamed.set_value(6, ['LandUse'], 'Transportation & Utility')
MN_BK_Merged_ColumnsRenamedDescribed7 = MN_BK_Merged_ColumnsRenamed.set_value(7, ['LandUse'], 'Public Facilities & Institutions')
MN_BK_Merged_ColumnsRenamedDescribed8 = MN_BK_Merged_ColumnsRenamed.set_value(8, ['LandUse'], 'Open Space & Outdoor Recreation')
MN_BK_Merged_ColumnsRenamedDescribed9 = MN_BK_Merged_ColumnsRenamed.set_value(9, ['LandUse'], 'Parking Facilities')
MN_BK_Merged_ColumnsRenamedDescribed10 = MN_BK_Merged_ColumnsRenamed.set_value(10, ['LandUse'], 'Vacant Land')
MN_BK_Merged_ColumnsRenamedDescribed10

,LandUse,bk,mn
0,One &Two Family Buildings,156271,4019
1,Multi-Family Walk-Up Buildings,66358,16246
2,Multi-Family Elevator Buildings,3437,4881
3,Mixed Residential & Commercial Buildings,22515,7238
4,Commercial & Office Buildings,5743,4465
5,Industrial & Manufacturing,4434,726
6,Transportation & Utility,1040,386
7,Public Facilities & Institutions,4330,2456
8,Open Space & Outdoor Recreation,872,434
9,Parking Facilities,4229,679


# create a feature (a column, a series) with the ratio of number of buildings in BK vs MN and find out in which LandUse class Manhattan has more buildings than BK 

In [19]:
# calculate the ratio of BK to Man buildings per LandUse category
# https://www.quora.com/How-do-I-divide-columns-in-pandas
MN_BK_Merged_ColumnsRenamedDescribed10['luratio'] = MN_BK_Merged_ColumnsRenamedDescribed10['mn']/MN_BK_Merged_ColumnsRenamedDescribed10['bk']
MN_BK_Merged_ColumnsRenamedDescribed10

,LandUse,bk,mn,luratio
0,One &Two Family Buildings,156271,4019,0.025718
1,Multi-Family Walk-Up Buildings,66358,16246,0.244824
2,Multi-Family Elevator Buildings,3437,4881,1.420134
3,Mixed Residential & Commercial Buildings,22515,7238,0.321475
4,Commercial & Office Buildings,5743,4465,0.777468
5,Industrial & Manufacturing,4434,726,0.163735
6,Transportation & Utility,1040,386,0.371154
7,Public Facilities & Institutions,4330,2456,0.567206
8,Open Space & Outdoor Recreation,872,434,0.497706
9,Parking Facilities,4229,679,0.160558


In [20]:
# defined function that returns True if the ratio is larger than 0, False otherwise. 
# I suppose to compare the ration to 1 instead of 0 from above...
# Use the series.apply() method to apply that function to the luratio series you jusrt created
# https://stackoverflow.com/questions/40221497/python-testing-true-false-condition-on-data-frame-column-and-returning-output-in
def ManhattanWins(luratio): 
    MN_BK_Merged_ColumnsRenamedDescribed10['Manhattan wins'] = np.where(MN_BK_Merged_ColumnsRenamedDescribed10['luratio']> 1, 'True', 'False')
#MN_BK_Merged_ColumnsRenamedDescribed10
    return ManhattanWins(MN_BK_Merged_ColumnsRenamedDescribed10)
MN_BK_Merged_ColumnsRenamedDescribed10

,LandUse,bk,mn,luratio
0,One &Two Family Buildings,156271,4019,0.025718
1,Multi-Family Walk-Up Buildings,66358,16246,0.244824
2,Multi-Family Elevator Buildings,3437,4881,1.420134
3,Mixed Residential & Commercial Buildings,22515,7238,0.321475
4,Commercial & Office Buildings,5743,4465,0.777468
5,Industrial & Manufacturing,4434,726,0.163735
6,Transportation & Utility,1040,386,0.371154
7,Public Facilities & Institutions,4330,2456,0.567206
8,Open Space & Outdoor Recreation,872,434,0.497706
9,Parking Facilities,4229,679,0.160558


# comment on what you found out. Does it make sense? 

After the computation and comparison, I found that only the ratio of Multi-Family Elevator Buildings in Manhattan is more than it in Brooklyn. However, it may not necessarily be true since this ratio is calculated by number of certain landuses of Manhattain divided by the number of certain landuses of Brooklyn. What we can only be sure is that the number of Multi-Family Elevator Buildings in Manhattan is more than it in Brooklyn. We all know that there are many high-rises in Manhattan and many of them are apartments, which fall into the multi-family elevator buildings category. While in Brooklyn there are more lower-level houses that do not have an elevator. So the result can be proved by our observations. As for the other types of landuses, Manhattan doesn't have as many of them as Brooklyn has probably due to the size of two boroughs. 